In [1]:
import import_ipynb
from OpenDataPandaFramework import *

importing Jupyter notebook from OpenDataPandaFramework.ipynb


The following parameters should be set to specify how many events will be read in every chunk, the directory for the files you would like to convert into hd5 files and a tag wich will be appended to the output filename to reflect the skimming applied

In [2]:
# The luminosity we run over
lumi = 10.6
# How many events to read in each round
chunksize = 500000
# How often to write to hd5 file (the smaller, the more files)
printevry = 500000
# Directory containing the input file (must end with .root)
#indir   = "/scratch3/eirikgr/openData_13TeV/Data"
indir   = "/scratch3/eirikgr/openData_13TeV/MC/SM_Backgrounds/"
# The output file name tag to store the skim options used abvove
datatype = indir.split("/")[-1]
# In cases there is a trailing / at the end
if not datatype: datatype = indir.split("/")[-2]
print("INFO \t Data type is {:s}".format(datatype))

# Define if it is signal or background (important for ML classification)
isMC = False
isData = False
isSignal = False
if datatype == "SM_Backgrounds":
    isMC = True
elif datatype == "BSM_Signal_Samples":
    isSignal = True
elif datatype == "Data":
    isData = True
else:
    print("ERROR \t Datatype {:s} is unknown. Setting as background".format(datatype))
    isMC = True
    
nlep = 2
lep_ptcut = [25000,25000]
skimtag = "2L_pt25_25_met50"

INFO 	 Data type is SM_Backgrounds


Specify the branches to keep in the hd5 files. If you like to check all the availale brances open a file and do events.keys()

In [3]:
branches = ['eventNumber','channelNumber',
            'mcWeight','scaleFactor_PILEUP','scaleFactor_ELE','scaleFactor_MUON',
            'scaleFactor_BTAG','scaleFactor_LepTRIGGER',
            'met_*','jet_*','lep_*']#'jet_n']#'jet_*']#,'photon_*','fatjet_*','tau_*','lep_*',
            #'ditau_m','truth_*']
#branches = ["*"]

In [4]:
res = OpenDataPandaFramework(indir,nlep, chunksize, printevry, datatype, skimtag, isMC, isData, isSignal, lep_ptcut, branches, lumi)

WARNING	 Result does not exists. Good :-)
INFO  	 Opening file 1/112: mc15_13TeV.410012.PwPyEG_P2012_singletop_tchan_lept_antitop.2lep_raw.root with 38934 events
INFO  	 ID 410012 in category Top has xsec = 26.3 fb and nev = 0.13 
INFO  	 Reading entries 0 - 38934 of 38934. Total so far: 0
WARNING	 Starting a new result panda
INFO  	 Read 38934 events in 1 files, for which 1567 (4.02%) were written to /scratch3/eirikgr/openData_13TeV/MC/SM_Backgrounds//SM_Backgrounds_2L_pt25_25_met50_num_1.h5


In [5]:
res

,eventNumber,channelNumber,met_et,met_phi,met_et_syst,wgt,isSignal,MCType,jet_n60,mll,lep1_pt,lep1_eta,lep1_phi,lep1_E,lep2_pt,lep2_eta,lep2_phi,lep2_E
entry,,,,,,,,,,,,,,,,,,
72,4720783,410012,6.029667e+04,2.536187,238.773438,0.000035,False,Top,1,46942.605469,3.650705e+04,-0.716060,1.549667,4.627320e+04,25597.310547,-2.117953,1.771689,107948.765625
87,4821831,410012,3.796668e+05,-2.675409,1124.000000,0.000074,False,Top,1,94836.664062,3.729063e+05,0.024953,0.543572,3.730224e+05,53906.253906,0.610407,0.234514,64264.628906
116,4721440,410012,5.419230e+04,-2.848535,7239.753906,0.000083,False,Top,0,68259.312500,3.941421e+04,0.046763,-0.613897,3.945732e+04,29839.115234,-0.571113,1.857708,34839.148438
118,4723055,410012,1.512745e+05,-0.193316,6484.421875,0.000072,False,Top,0,122424.257812,1.421722e+05,-0.071337,2.701859,1.425342e+05,35068.675781,1.082576,-2.252842,57705.464844
132,605225,410012,1.425279e+05,0.031598,942.640625,0.000020,False,Top,1,39343.484375,9.692611e+04,0.034899,2.952590,9.698520e+04,32899.742188,0.667835,2.684862,40513.214844
134,2736026,410012,5.546291e+04,-1.464220,9017.671875,0.000072,False,Top,0,25262.433594,5.758579e+04,0.146611,2.165090,5.820579e+04,27748.861328,-0.011068,1.543177,27750.761719
165,606654,410012,6.113518e+04,0.619287,373.675781,-0.000088,False,Top,0,154843.500000,3.604218e+04,-0.414706,-1.021874,3.918614e+04,32650.613281,2.553308,0.998784,211043.203125
185,4724475,410012,1.423524e+05,0.986140,30829.132812,0.000072,False,Top,0,237452.921875,1.421115e+05,-0.054388,-2.149539,1.423218e+05,74742.492188,-1.570068,-0.307341,187416.937500
190,4742955,410012,1.014629e+05,1.851679,568.929688,-0.000037,False,Top,1,87238.265625,8.425402e+04,-1.910092,0.443975,2.907513e+05,25552.894531,-1.241258,2.579685,47898.695312
